**Dados de casos de COVID 19 no Brasil por município e estado**

Libraries

In [0]:
import pandas as pd
import numpy as np

###Leitura de CSV's

Base de dados do Brasil IO

In [0]:
data_brasil = pd.read_csv('https://data.brasil.io/dataset/covid19/caso.csv.gz', compression='gzip', header=0, quotechar='"', error_bad_lines=False)
data_brasil.rename(columns={"city_ibge_code":"codigo_ibge","state":"sigla_state","estimated_population_2019":"population"},inplace = True)
data_brasil.head(2)

,date,sigla_state,city,place_type,confirmed,deaths,order_for_place,is_last,population,codigo_ibge,confirmed_per_100k_inhabitants,death_rate
0,2020-04-01,SC,NaN,state,247,2.0,21,True,7164788.0,42.0,3.44742,0.0081
1,2020-03-31,SC,NaN,state,235,2.0,20,False,7164788.0,42.0,3.27993,0.0085


Dados de Lagitude e Longitude dos municípios 

In [0]:
city_lat_long =  pd.read_csv("https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/master/csv/municipios.csv")
city_lat_long.drop(columns = ["capital"], inplace = True)
city_lat_long.rename(columns = {"nome":"city"},inplace = True)
city_lat_long.head(2)

,codigo_ibge,city,latitude,longitude,codigo_uf
0,5200050,Abadia de Goiás,-16.7573,-49.4412,52
1,3100104,Abadia dos Dourados,-18.4831,-47.3916,31


Base do IBGE, contendo código do município, nome do estado, do município, e o código do estado

In [0]:
data_ibge = pd.read_excel("https://github.com/vilsonrodrigues/COVID19/blob/master/RELATORIO_DTB_BRASIL_MUNICIPIO.xls?raw=true")
data_ibge.drop(columns = ["Mesorregião Geográfica","Nome_Mesorregião","Microrregião Geográfica","Nome_Microrregião","Município"], inplace = True)
data_ibge.rename(columns = {"Nome_Município":"city","Nome_UF":"state","UF":"codigo_uf","Código Município Completo":"codigo_ibge"},inplace = True)
data_ibge.head(2)

,codigo_uf,state,codigo_ibge,city
0,11,Rondônia,1100015,Alta Floresta D'Oeste
1,11,Rondônia,1100379,Alto Alegre dos Parecis


###Manipulação de dataframes

Juntando a tabela da localização geográfica com a dos dados do ibge

In [0]:
city_lat_long = city_lat_long.merge(data_ibge, on = ["city","codigo_ibge","codigo_uf"],sort = "city")
city_lat_long.head(2)

,codigo_ibge,city,latitude,longitude,codigo_uf,state
0,5200050,Abadia de Goiás,-16.7573,-49.4412,52,Goiás
1,3100104,Abadia dos Dourados,-18.4831,-47.3916,31,Minas Gerais


Merge de tabelas de caso e localização geográfica


In [0]:
data_brasil = data_brasil.merge(city_lat_long, on = ["city","codigo_ibge"],sort = "date",how="outer")
data_brasil.dropna(subset=["date"],inplace=True)
data_brasil.head(2)

,date,sigla_state,city,place_type,confirmed,deaths,order_for_place,is_last,population,codigo_ibge,confirmed_per_100k_inhabitants,death_rate,latitude,longitude,codigo_uf,state
3,2020-04-01,PA,Abaetetuba,city,1.0,0.0,2.0,True,157698.0,1500107.0,0.63412,NaN,-1.72183,-48.8788,15.0,Pará
4,2020-03-31,PA,Abaetetuba,city,1.0,0.0,1.0,False,157698.0,1500107.0,0.63412,NaN,-1.72183,-48.8788,15.0,Pará


Retirando valores nulos e passando o codigo do ibge pra inteiro

In [0]:
data_brasil.replace(np.nan, 0, inplace = True)
data_brasil["codigo_ibge"] = data_brasil["codigo_ibge"].astype("int64")
data_brasil.head(2)

,date,sigla_state,city,place_type,confirmed,deaths,order_for_place,is_last,population,codigo_ibge,confirmed_per_100k_inhabitants,death_rate,latitude,longitude,codigo_uf,state
3,2020-04-01,PA,Abaetetuba,city,1.0,0.0,2.0,True,157698.0,1500107,0.63412,0.0,-1.72183,-48.8788,15.0,Pará
4,2020-03-31,PA,Abaetetuba,city,1.0,0.0,1.0,False,157698.0,1500107,0.63412,0.0,-1.72183,-48.8788,15.0,Pará


Dataset com os ultimos boletins

In [0]:
data_brasil_last_boletim = data_brasil.loc[data_brasil["is_last"] == True,:]
data_brasil_last_boletim.head(2)

,date,sigla_state,city,place_type,confirmed,deaths,order_for_place,is_last,population,codigo_ibge,confirmed_per_100k_inhabitants,death_rate,latitude,longitude,codigo_uf,state
3,2020-04-01,PA,Abaetetuba,city,1.0,0.0,2.0,True,157698.0,1500107,0.63412,0.0,-1.72183,-48.8788,15.0,Pará
26,2020-04-01,AC,Acrelândia,city,8.0,0.0,4.0,True,15256.0,1200013,52.43838,0.0,-9.82581,-66.8972,12.0,Acre


###Download CSV's

In [0]:
data_brasil.to_csv("data_brasil.csv")

In [0]:
data_brasil_last_boletim.to_csv("data_brasil_last_boletim.csv")